In [1]:
using SymPy

In [2]:
function _sym(x::Expr)
    y = x
    y.args .= _sym.(y.args)
    return  y
end

_sym(x::Number) = Sym(x)
_sym(x) = x

macro sym(x)
    return esc(_sym(x))
end

@sym (macro with 1 method)

In [3]:
function get_glrk_nodes(s)
    if s == 1
        c = @sym Vector([1//2])
    elseif s == 2
        c = @sym [1/2-√3/6, 1/2+√3/6]
    elseif s == 3
        c = @sym [1/2-√15/10,  1/2,        1/2+√15/10 ]
    elseif s == 4
        c = @sym [
          1/2-√(3/7+2*√30/35)/2,
          1/2-√(3/7-2*√30/35)/2,
          1/2+√(3/7-2*√30/35)/2,
          1/2+√(3/7+2*√30/35)/2
        ]
    else
        @error "GLRK nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_glrk_nodes (generic function with 1 method)

In [4]:
function get_lobatto_nodes(s)
    if s == 2
        c = @sym [0, 1]
    elseif s == 3
        c = @sym [0, 1//2, 1]
    elseif s == 4
        c = @sym [0,  (5-√5)/10,   (5+√5)/10,  1]
    elseif s == 5
        c = @sym [0,  1//2-√21/14,  1//2,  1//2+√21/14,  1]
    else
        @error "Lobatto nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_lobatto_nodes (generic function with 1 method)

In [27]:
function get_glrk_lobatto_tableau(s)
    c = get_glrk_nodes(s)
    c̄ = get_lobatto_nodes(s+1)
    a = [Sym("a_" * string(i) * string(j)) for i in 0:s, j in 1:s]
    A = [Sym(0) for i in 0:s, j in 1:s]
    
    sol = Dict()

    for i in 0:s
        eqs = Sym[]
        for k in 1:s
            eq = - c̄[i+1]^k / k
            for j in 1:s
                eq += a[i+1,j] * c[j]^(k-1)
            end
            push!(eqs, eq)
        end

        for k in 1:s
            A[i+1,k] = solve(eqs[k], a[i+1,k])[1]            
            for l in k+1:s
                eqs[l] = simplify(subs(eqs[l], a[i+1,k], A[i+1,k]))
            end
        end
        
        for k in s-1:-1:1
            for l in k+1:s
                A[i+1,k] = subs(A[i+1,k], a[i+1,l], A[i+1,l])
            end
#            A[i+1,k] = simplify(A[i+1,k])
        end
    end
    
    return A
end

get_glrk_lobatto_tableau (generic function with 1 method)

In [28]:
get_glrk_lobatto_tableau(1)

2×1 Array{Sym,2}:
 0
 1

In [29]:
get_glrk_lobatto_tableau(2)

3×2 Array{Sym,2}:
               0                 0
 sqrt(3)/8 + 1/4  -sqrt(3)/8 + 1/4
             1/2               1/2

In [30]:
get_glrk_lobatto_tableau(3)

4×3 Array{Sym,2}:
                                 0  …                                  0
 -sqrt(5)/180 + sqrt(15)/30 + 5/36     -sqrt(15)/30 - sqrt(5)/180 + 5/36
  sqrt(5)/180 + sqrt(15)/30 + 5/36     -sqrt(15)/30 + sqrt(5)/180 + 5/36
                              5/18                                  5/18

In [39]:
a = get_glrk_lobatto_tableau(4)
b = N.(a)

5×4 Array{Real,2}:
 0         0           0            0
 0.159167  0.0156555  -0.00264993   0.000500452
 0.176106  0.304916    0.0211566   -0.00217852
 0.173427  0.328723    0.310417     0.0147603
 0.173927  0.326073    0.326073     0.173927

In [50]:
function array_to_code(name, x::Matrix)
    s = string(name) * " = [\n"
    
    for i in axes(x,1)
        s *= "["
        for j in axes(x,2)
            s *= " "
            s *= string(x[i,j])
            s *= " "
        end
        s *= "]\n"
    end
    
    s *= "]\n"
end

array_to_code (generic function with 1 method)

In [52]:
array_to_code(:x, b)

"x = [\n[ 0  0  0  0 ]\n[ 0.1591671294900345358547852036503968244037099929876660089701433805420016628303049  0.01565551541810189985834126767767053597372403599533987485875443079743596295789264  -0.002649931830622319490548503812025451589560291885905677307276227674989121280824921  0.0005004515684973118782758043605289134275222217051875147207182646279180365249293946 ]\n[ 0.176105937938662021225385981377510389121610166295011341406551070207182545793024  0.3049159394838621526624258370570061164298248408582482915516281042450345033436905  0.02115663794741091865104218833199417995250081122480493820210723599558956292335403  -0.002178515369935092538854006766510685503935818378064571160286410447806612060067542 ]\n[ 0.1734269710002296168082561702504707901901521262117592555255463951314578972080256  0.3287225092618953908040165292010257479718859439689589070610115679156131875478712  0.3104170620131711714551267577113297604086016160877133548949809094431881033091524  0.01476029307869239283174677096060287921396

In [53]:
println(array_to_code(:x, b))

x = [
[ 0  0  0  0 ]
[ 0.1591671294900345358547852036503968244037099929876660089701433805420016628303049  0.01565551541810189985834126767767053597372403599533987485875443079743596295789264  -0.002649931830622319490548503812025451589560291885905677307276227674989121280824921  0.0005004515684973118782758043605289134275222217051875147207182646279180365249293946 ]
[ 0.176105937938662021225385981377510389121610166295011341406551070207182545793024  0.3049159394838621526624258370570061164298248408582482915516281042450345033436905  0.02115663794741091865104218833199417995250081122480493820210723599558956292335403  -0.002178515369935092538854006766510685503935818378064571160286410447806612060067542 ]
[ 0.1734269710002296168082561702504707901901521262117592555255463951314578972080256  0.3287225092618953908040165292010257479718859439689589070610115679156131875478712  0.3104170620131711714551267577113297604086016160877133548949809094431881033091524  0.0147602930786923928317467709606028792139643549